### Импорты

transliterate

langchain???

langchain-text-splitters

langchain-community

requests

langchain-chroma

In [1]:
import os
import time
from transliterate import translit

from mediawikiapi import MediaWikiApi
from documentprocessor import DocumentProcessor
from vectorstorebuilder import VectorStoreBuilder
from permianragsystem import PermianRAGSystem

In [11]:
mediawikiapi = MediaWikiApi('ru')

In [12]:
terms = [
    "Пермский период",
    "Пермское вымирание",
    "Пермско-триасовое вымирание",
    "Поздний палеозой",
    "Палеозойская эра",
    "Пермская система",
]

In [ ]:
all_titles = set()

for term in terms:
    print(f'Поиск по запросу: {term}')
    try:
        term_titles = mediawikiapi.get_titles(term, 10)
        all_titles.update(term_titles)
        time.sleep(0.5)
    except Exception as e:
        print(f'Ошибка при поиске по запросу "{term}": {e}')

Поиск по запросу: Пермский период
Поиск по запросу: Пермское вымирание
Поиск по запросу: Пермско-триасовое вымирание
Поиск по запросу: Поздний палеозой
Поиск по запросу: Палеозойская эра
Поиск по запросу: Пермская система


In [14]:
all_titles

{'3I/ATLAS',
 'Архозавры',
 'Верхнекамское месторождение калийно-магниевых солей',
 'Вымирание',
 'Геологическая история Земли',
 'Геология Антарктиды',
 'Герцинская складчатость',
 'Гибель человечества',
 'Ильмень',
 'История Земли',
 'История жизни на Земле',
 'Кембрийский период',
 'Кратер Земли Уилкса',
 'Куштау',
 'Ледниковый период',
 'Массовое вымирание',
 'Массовое пермское вымирание',
 'Мел-палеогеновое вымирание',
 'Ордовикско-силурийское вымирание',
 'Палеозой',
 'Палеонтологический музей имени Ю. А. Орлова',
 'Пангея',
 'Пермская ГРЭС',
 'Пермская область',
 'Пермский край',
 'Пермский период',
 'Пермь',
 'Растения',
 'Салават',
 'Сибирские траппы',
 'Силурийский период',
 'Союз Советских Социалистических Республик',
 'Татарский отдел',
 'Триасово-юрское вымирание',
 'Триасовый период',
 'Шахтау',
 'Шихан',
 'Энциклопедический словарь Брокгауза и Ефрона',
 'Эра',
 'Юрактау'}

In [17]:
non_relevant_titles = [
    '3I/ATLAS',
    'Гибель человечества',
    'Ильмень',
    'Палеонтологический музей имени Ю. А. Орлова',
    'Пермская ГРЭС',
    'Пермская область',
    'Пермский край',
    'Пермь',
    'Растения',
    'Салават',
    'Силурийский период',
    'Союз Советских Социалистических Республик',
    'Татарский отдел',
    'Триасово-юрское вымирание',
    'Энциклопедический словарь Брокгауза и Ефрона',
    'Эра',
]

In [18]:
relevant_titles = [title for title in all_titles if title not in non_relevant_titles]

In [19]:
relevant_titles

['Сибирские траппы',
 'Вымирание',
 'Архозавры',
 'Массовое вымирание',
 'Кратер Земли Уилкса',
 'Герцинская складчатость',
 'Массовое пермское вымирание',
 'Ордовикско-силурийское вымирание',
 'Геологическая история Земли',
 'Палеозой',
 'Шихан',
 'Кембрийский период',
 'Триасовый период',
 'Мел-палеогеновое вымирание',
 'Пермский период',
 'Ледниковый период',
 'Куштау',
 'Пангея',
 'Юрактау',
 'Шахтау',
 'Верхнекамское месторождение калийно-магниевых солей',
 'История Земли',
 'История жизни на Земле',
 'Геология Антарктиды']

In [20]:
data = []

for title in relevant_titles:
    print(f'Поиск текста по статье: {title}')
    try:
        text = mediawikiapi.get_text(title)
        data.append({
            'title': title,
            'text': text,
        })
        time.sleep(0.5)
    except Exception as e:
        print(f'Ошибка при поиске текста статьи "{title}": {e}')

Поиск текста по статье: Сибирские траппы
Поиск текста по статье: Вымирание
Поиск текста по статье: Архозавры
Поиск текста по статье: Массовое вымирание
Поиск текста по статье: Кратер Земли Уилкса
Поиск текста по статье: Герцинская складчатость
Поиск текста по статье: Массовое пермское вымирание
Поиск текста по статье: Ордовикско-силурийское вымирание
Поиск текста по статье: Геологическая история Земли
Поиск текста по статье: Палеозой
Поиск текста по статье: Шихан
Поиск текста по статье: Кембрийский период
Поиск текста по статье: Триасовый период
Поиск текста по статье: Мел-палеогеновое вымирание
Поиск текста по статье: Пермский период
Поиск текста по статье: Ледниковый период
Поиск текста по статье: Куштау
Поиск текста по статье: Пангея
Поиск текста по статье: Юрактау
Поиск текста по статье: Шахтау
Поиск текста по статье: Верхнекамское месторождение калийно-магниевых солей
Поиск текста по статье: История Земли
Поиск текста по статье: История жизни на Земле
Поиск текста по статье: Геоло

In [ ]:
output_dir = "./permian_articles"
os.makedirs(output_dir, exist_ok=True)

for article in data:
    safe_article = article['title'].lower().replace(' ', '_').replace('-', '_')
    translit_safe_article = translit(safe_article, language_code='ru', reversed=True)
    filename = f"{translit_safe_article}.txt"
    filepath = os.path.join(output_dir, filename)

    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(article['text'])

    print(f'Статья "{article['title']}" сохранена по пути {filepath}')

Статья "Сибирские траппы" сохранена по пути permian_articles\sibirskie_trappy.txt
Статья "Вымирание" сохранена по пути permian_articles\vymiranie.txt
Статья "Архозавры" сохранена по пути permian_articles\arhozavry.txt
Статья "Массовое вымирание" сохранена по пути permian_articles\massovoe_vymiranie.txt
Статья "Кратер Земли Уилкса" сохранена по пути permian_articles\krater_zemli_uilksa.txt
Статья "Герцинская складчатость" сохранена по пути permian_articles\gertsinskaja_skladchatost'.txt
Статья "Массовое пермское вымирание" сохранена по пути permian_articles\massovoe_permskoe_vymiranie.txt
Статья "Ордовикско-силурийское вымирание" сохранена по пути permian_articles\ordoviksko_silurijskoe_vymiranie.txt
Статья "Геологическая история Земли" сохранена по пути permian_articles\geologicheskaja_istorija_zemli.txt
Статья "Палеозой" сохранена по пути permian_articles\paleozoj.txt
Статья "Шихан" сохранена по пути permian_articles\shihan.txt
Статья "Кембрийский период" сохранена по пути permian_art

In [2]:
documentprocessor = DocumentProcessor('./permian_articles')
stats = documentprocessor.load_and_chunk()
chunks = documentprocessor.chunks

In [3]:
stats

{'num_documents': 24,
 'num_chunks': 642,
 'total_chars': 393252,
 'avg_chunk_size': 633.588785046729}

In [2]:
vectorstorebuilder = VectorStoreBuilder('zylonai/multilingual-e5-large:latest')

if vectorstorebuilder.exists():
    vectorstore = vectorstorebuilder.load()
else:
    vectorstore = vectorstorebuilder.build(chunks)

In [ ]:
# Определяем набор контрольных вопросов, категорий и ожидаемых ключевых слов
questions = [
    # 1. Хронология
    {
        "question": "Сколько продолжался Пермский период и каковы его точные даты начала и окончания?",
        "category": "Хронология",
        "expected_keywords": ["298,9", "251", "миллион", "лет", "начало", "окончание", "палеозой"]
    },

    # 2. Климат и география
    {
        "question": "Каким был климат на Земле во время Пермского периода? Опишите основные тенденции.",
        "category": "Климат и география",
        "expected_keywords": ["теплый", "холодный", "оледенение", "засушливый", "пустыня", "суперконтинент", "Пангея"]
    },
    {
        "question": "Как выглядела география Земли в Пермском периоде? Какой суперконтинент существовал?",
        "category": "Климат и география",
        "expected_keywords": ["Пангея", "океан", "Палеотетис", "суперконтинент", "расположение"]
    },

    # 3. Причины вымирания
    {
        "question": "Что стало основной причиной Пермско-триасового вымирания, самого масштабного в истории Земли?",
        "category": "Вымирание",
        "expected_keywords": ["сибирские траппы", "вулканизм", "базальт", "метан", "парниковый эффект", "аноксия"]
    },
    {
        "question": "Какие гипотезы, кроме вулканической активности, существуют для объяснения Великого вымирания?",
        "category": "Вымирание",
        "expected_keywords": ["метеорит", "аноксия", "закисление", "океан", "климат"]
    },

    # 4. Доминирующие животные
    {
        "question": "Какие наземные позвоночные (рептилии) были доминирующими видами в Пермском периоде?",
        "category": "Фауна",
        "expected_keywords": ["терапсиды", "зверообразные", "горгонопсы", "диноцефалы", "дицинодонты", "пеликозавры"]
    },
    {
        "question": "Опишите разнообразие и экологические ниши синапсид в Пермском периоде.",
        "category": "Фауна",
        "expected_keywords": ["синапсиды", "хищники", "травоядные", "терапсиды", "пеликозавры"]
    },

    # 5. Влияние на эволюцию жизни
    {
        "question": "Как Пермское вымирание повлияло на дальнейшую эволюцию жизни на Земле? Какие группы выжили, а какие исчезли?",
        "category": "Эволюция",
        "expected_keywords": ["архозавры", "предки млекопитающих", "терапсиды", "восстановление", "мезозой"]
    },
    {
        "question": "Какие эволюционные инновации появились у животных в течение Пермского периода?",
        "category": "Эволюция",
        "expected_keywords": ["дифференциация зубов", "вторичное нёбо", "постановка конечностей"]
    },

    # 6. Условия в морях
    {
        "question": "Какими были условия в морях и океанах Пермского периода? Опишите соленость, температуру и уровень кислорода.",
        "category": "Морская среда",
        "expected_keywords": ["рифы", "брахиоподы", "аммониты", "фузулиниды", "соленость", "температура"]
    },
    {
        "question": "Какие организмы доминировали в морских экосистемах Пермского периода?",
        "category": "Морская среда",
        "expected_keywords": ["брахиоподы", "аммониты", "кораллы", "губки", "фузулиниды"]
    },

    # 7. Растительность
    {
        "question": "Какие основные группы растений существовали на суше в Пермском периоде?",
        "category": "Флора",
        "expected_keywords": ["голосеменные", "хвойные", "гинкговые", "саговники", "папоротники", "хвощи"]
    },
    {
        "question": "Как изменялась растительность от начала к концу Пермского периода?",
        "category": "Флора",
        "expected_keywords": ["лес", "гинкго", "хвойные", "упрощение", "засушливость"]
    },

    # 8. Атмосфера
    {
        "question": "Какой был состав атмосферы Земли в Пермском периоде? Как менялось содержание кислорода и углекислого газа?",
        "category": "Атмосфера",
        "expected_keywords": ["кислород", "углекислый газ", "CO2", "метан", "парниковый эффект", "уровень"]
    },
    {
        "question": "Какое влияние изменения атмосферы оказали на климат и живые организмы Пермского периода?",
        "category": "Атмосфера",
        "expected_keywords": ["потепление", "похолодание", "оледенение", "дыхание", "адаптация"]
    }
]

In [3]:
rag_system = PermianRAGSystem(vectorstore, 'qwen2.5:7b')

In [ ]:
question = "Как изменялась растительность от начала к концу Пермского периода?"

# Stream the answer token by token
for token in rag_system.steam_answer_question(question):
    print(token, end="", flush=True)
    time.sleep(0.05)

Сначала в конце карбона — начале пермского периода климат стал прохладнее и суше, пышные тропические леса и болота сменились пустынями и засушливыми равнинами. В таких условиях росли только самые стойкие растения — папоротники и примитивные хвойные.

В середине пермского периода климат Европы и Северной Америки стал более засушливым, что привело к исчезновению папоротников, каламитов, древовидных плаунообразных и других влаголюбивых растений тропического леса. Лишь в восточных районах Лавразии климат и флора оставались близкими к таковым в каменноугольное время.

Таким образом, от начала к концу Пермского периода растительность сначала адаптировалась к более суровым условиям, сохраняя папоротники и примитивные хвойные, а затем постепенно исчезали типичные для тропических лесов виды растений.

In [ ]:
results = []

for q in questions:
    result = rag_system.answer_question(q["question"])
    result["category"] = q["category"]
    result["expected_keywords"] = q["expected_keywords"]
    results.append(result)
    print(f"\n{'='*80}")
    print(f"Q: {result['question']}")
    print(f"{'─'*80}")
    print(result['answer'])


QСколько продолжался Пермский период и каковы его точные даты начала и окончания?
────────────────────────────────────────────────────────────────────────────────
Пермский период начался примерно 298,9 миллиона лет назад и закончился около 251,902 миллиона лет назад. Этот период продолжался около 47 миллионов лет.

В более точной форме:
Начало: 298,9 ± 0,15 млн лет назад
Окончание: 251,902 ± 0,024 млн лет назад
Длительность: около 47 млн лет

QНа какие геологические эпохи подразделяется Пермский период?
────────────────────────────────────────────────────────────────────────────────
Это информация не доступна в предоставленных документах.

QКаким был климат на Земле во время Пермского периода? Опишите основные тенденции.
────────────────────────────────────────────────────────────────────────────────
Климат Пермского периода характеризовался резко выраженной зональностью и увеличивающейся сушностью. В целом он был похож на современный климат, что отличало его от последующих периодов м